# Transformaciones de Datos

Usualmente no se tiene los datos en un formato conveniente. Una gran parte del trabajo con datos consiste en usar el conocimiento de un dominio determinado para saber cómo manejar los datos (eliminar algunos datos faltantes, realizar "feature engineering", transformar datos, etc.)

Spark tiene métodos para realizar estas transformaciones: http://spark.apache.org/docs/latest/ml-features.html

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('EjemploTransformDatos').getOrCreate()

## Conversión de atributos categóricos a numéricos

`StringIndexer` se utiliza para convertir atributos categóricos (no numéricos) en atributos numéricos.

In [3]:
from pyspark.ml.feature import StringIndexer

In [4]:
# Se crea un dataset de ejemplo, donde "categoría" es no numérico
df = spark.createDataFrame([(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],
                           ["ID", "categoria"])
df.show()

+---+---------+
| ID|categoria|
+---+---------+
|  0|        a|
|  1|        b|
|  2|        c|
|  3|        a|
|  4|        a|
|  5|        c|
+---+---------+



In [5]:
# Con StringIndexer se indica que se va a convertir la columna "categoria" en numérica ("índices")
indexer = StringIndexer(inputCol="categoria", outputCol="IndiceCategoria")

# Ajustar el mapa entre categorías y valores numéricos (índices)
df2 = indexer.fit(df)

# Mostrar las etiquetas que se mapean como (0, 1, 2)
df2.labels

['a', 'c', 'b']

In [6]:
# Transformar los datos según los índices generados
df2 = df2.transform(df)
df2.show()

+---+---------+---------------+
| ID|categoria|IndiceCategoria|
+---+---------+---------------+
|  0|        a|            0.0|
|  1|        b|            2.0|
|  2|        c|            1.0|
|  3|        a|            0.0|
|  4|        a|            0.0|
|  5|        c|            1.0|
+---+---------+---------------+



## Generación de un vector columna (combinando otras columnas)

`VectorAssembler` combina un conjunto de columnas en un solo vector columna. Es útil para combinar atributos originales con aquellos generados por diferentes transformaciones aplicadas en PySpark. Esto es necesario para tener el formato que los modelos de ML de Spark utilizan. 

`VectorAssembler` acepta los siguientes tipos de columnas: todos los tipos numéricos, tipos Booleanos, tipos vector. En cada fila, los valores de las columnas de entrada serán concatenados en un vector de un orden especificado.

Ejemplo: Si se tiene un DataFrame con las columnas id, campo1, campo2, campos3, y valor:

     id | campo1 | campo2 |   campos3   | valor
    ----|--------|--------|-------------|------
    204 |   18   |   1.0  | [3, 10, 20] |  5.9
     
donde `campos3` es una columna de vectores que contiene tres atributos. Se desea combinar `campo1`, `campo2` y `campos3` en un solo vector de atributos llamado `vatributos` para ser usado como predictor de `valor`. Si se indica que las columnas de entrada de `VectorAssembler` son `campo1`, `campo2` y `campos3`, y que la columna de salida es `valor`, luego de la transformación se obtendrá lo siguiente:

     id | campo1 | campo2 |   campos3   | valor |     vatributos
    ----|--------|--------|-------------|-------|----------------------
    204 |   18   |   1.0  | [3, 10, 20] |  5.9  | [18, 1.0, 3, 10, 20]

In [7]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors

In [8]:
# Vector denso en Spark
vector = Vectors.dense([3, 10, 20])
vector

DenseVector([3.0, 10.0, 20.0])

In [9]:
# Creación de un data frame (de una fila)
df = spark.createDataFrame([(204, 18, 1.0, vector, 5.9),
                            (205, 25, 3.5, vector, 6.7)],
                           ["id", "campo1", "campo2", "campos3", "valor"])
df.show()

+---+------+------+---------------+-----+
| id|campo1|campo2|        campos3|valor|
+---+------+------+---------------+-----+
|204|    18|   1.0|[3.0,10.0,20.0]|  5.9|
|205|    25|   3.5|[3.0,10.0,20.0]|  6.7|
+---+------+------+---------------+-----+



In [10]:
# Objeto que juntará columnas para crear una sola columna
assembler = VectorAssembler(inputCols=["campo1", "campo2", "campos3"],
                            outputCol="vatributos")

# Transformar los datos según la columna creada
df2 = assembler.transform(df)
df2.show()

+---+------+------+---------------+-----+--------------------+
| id|campo1|campo2|        campos3|valor|          vatributos|
+---+------+------+---------------+-----+--------------------+
|204|    18|   1.0|[3.0,10.0,20.0]|  5.9|[18.0,1.0,3.0,10....|
|205|    25|   3.5|[3.0,10.0,20.0]|  6.7|[25.0,3.5,3.0,10....|
+---+------+------+---------------+-----+--------------------+



In [11]:
df2.show(truncate=False)

+---+------+------+---------------+-----+------------------------+
|id |campo1|campo2|campos3        |valor|vatributos              |
+---+------+------+---------------+-----+------------------------+
|204|18    |1.0   |[3.0,10.0,20.0]|5.9  |[18.0,1.0,3.0,10.0,20.0]|
|205|25    |3.5   |[3.0,10.0,20.0]|6.7  |[25.0,3.5,3.0,10.0,20.0]|
+---+------+------+---------------+-----+------------------------+



In [12]:
# Seleccionar solo las columnas vatributos y valor (usual como entrada a algoritmos supervizados)
df2.select("vatributos", "valor").show()

+--------------------+-----+
|          vatributos|valor|
+--------------------+-----+
|[18.0,1.0,3.0,10....|  5.9|
|[25.0,3.5,3.0,10....|  6.7|
+--------------------+-----+

